<h1 style="text-align: center;">Tim "Poslednji ispit" - University of Kragujevac</h1>

<p style="text-align: center;">
  <img src="logopmfy.png" alt="University of Kragujevac Logo" width="200" height="200">
</p>

<h2 style="text-align: center;">Team Members</h2>

<ul style="list-style-type: none; text-align: center;">
  <li><strong>Vuk Lazović</strong></li>
  <li><strong>Sara Verlimirović</strong></li>
  <li><strong>Mihajlo Janković</strong></li>
</ul>


# Predstavljanje problema

.....

# Preuzimanje dataseta

In [ ]:
pip install -r requirements.txt

In [23]:
import requests,os
from tqdm import tqdm
import pandas as pd
import numpy as np

In [1]:
def check_dataset():
    files = os.listdir('.')
    flag=1
    for file in files:
        if("vehicles.csv"==file):
            flag=0
    return flag

def download():
    print("Downloading dataset.....")
    url = "https://www.dropbox.com/scl/fi/hiod02ra6fa1d5f5q7bmd/vehicles.csv?rlkey=ein4k3paqkw0ashh8njtyg6ed&st=gvyd3ohd&dl=1"
    response = requests.get(url, stream=True)
    
    # Get the total file size
    total_size = int(response.headers.get('content-length', 0))
    
    with open("vehicles.csv", mode="wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=10 * 1024), total=total_size//(10*1024), unit='KB'):
            file.write(chunk)

    response = requests.get(url, stream=True)


if(check_dataset()):
    download()
else:
    print("Dataset already downloaded")

Dataset already downloaded


In [3]:
df = pd.read_csv("vehicles.csv")
df


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [4]:
list(df.columns)

['id',
 'url',
 'region',
 'region_url',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'VIN',
 'drive',
 'size',
 'type',
 'paint_color',
 'image_url',
 'description',
 'county',
 'state',
 'lat',
 'long',
 'posting_date']

In [5]:
df['manufacturer']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
426875      nissan
426876       volvo
426877    cadillac
426878       lexus
426879         bmw
Name: manufacturer, Length: 426880, dtype: object

In [13]:
pip install Jinja2


   ---------------------------------------- 0.0/133.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.3 kB ? eta -:--:--
   ----------- --------------------------- 41.0/133.3 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 133.3/133.3 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Uopstena analiza dataset-a. 

# Sažetak za numeričke kolone
numerical_summary = df.describe().round(2)

# Sažetak za kategorijske kolone (ne-numeričke)
categorical_summary = df.select_dtypes(exclude='number').describe()

# Pregled strukture DataFrame-a
data_structure = df.info()

# Prikazivanje nekoliko prvih vrednosti za pregled
head_values = df.head()

# Kod stringova postoji potencijalni problem praznih stringova, 
# zato cemo sve prazne stringove zameniti sa NaN vrednostima, 
# a nakon toga cemo posmatrati NaN vrednosti.
df.replace("", np.nan, inplace=True)

# Broj NaN vrednosti po kolonama
nan_count = df.isna().sum()

# Broj ne NaN vrednosti
non_nan_counts = df.notna().sum()

# Ispis rezultata
print("Sažetak za numeričke kolone:")
print(numerical_summary)
print("\nSažetak za kategorijske kolone:")
print(categorical_summary)
print("\nPregled strukture DataFrame-a:")
print(data_structure)
print("\nPrvih nekoliko vrednosti u DataFrame-u:")
print(head_values)

# Primecujemo da postoji odredjeni broj NaN vrednosti po kolonama
print("\n Nan vrednosti")
print(nan_count)

print("\n Ne Nan vrednosti")
print(non_nan_counts)

#print(non_nan_counts - nan_count)

#bmw = df[df['manufacturer'] == 'bmw']



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [20]:
dfz=bmw[['year','manufacturer','model','odometer','price']].dropna()

In [24]:
len(dfz)

14494

In [22]:
from sklearn.model_selection import train_test_split

df_encoded = pd.get_dummies(dfz, columns=['manufacturer','model'])
y=df_encoded['price']
x=df_encoded.drop('price', axis=1)
train_df, test_df = train_test_split(x, test_size=0.2, random_state=42)
train_y, test_y = train_test_split(y, test_size=0.2, random_state=42)


In [33]:
test_y

335211    12695
63042     33590
313859     4000
136666    24980
6484      27995
          ...  
276067    21990
347902    12999
255292    35885
304907     5500
123423        0
Name: price, Length: 2899, dtype: int64

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create a Random Forest regressor
rf_regressor = RandomForestRegressor(n_estimators= 100, random_state=42)

# Train the model
rf_regressor.fit(train_df, train_y)

# Make predictions on the test set
y_pred = rf_regressor.predict(test_df)

# Calculate Mean Squared Error
mse = mean_squared_error(test_y, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# Calculate R-squared
r2 = r2_score(test_y, y_pred)
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 39132401.87
R-squared: 0.79


In [38]:
for i,j in zip(test_y,y_pred):
    print(f"actual : {i}\t predicted : {j}")

actual : 12695	 predicted : 13981.68
actual : 33590	 predicted : 36175.02547085568
actual : 4000	 predicted : 5101.2
actual : 24980	 predicted : 24800.15
actual : 27995	 predicted : 23387.27
actual : 27500	 predicted : 27419.9
actual : 0	 predicted : 0.0
actual : 15995	 predicted : 15036.74
actual : 17995	 predicted : 18051.890833333324
actual : 28990	 predicted : 28990.0
actual : 26500	 predicted : 29415.38
actual : 0	 predicted : 8552.93
actual : 14590	 predicted : 14590.0
actual : 21850	 predicted : 21904.2
actual : 24990	 predicted : 24990.0
actual : 21688	 predicted : 21688.0
actual : 12950	 predicted : 12886.0
actual : 28400	 predicted : 28400.0
actual : 24000	 predicted : 20958.36
actual : 36590	 predicted : 36590.0
actual : 4500	 predicted : 6839.23
actual : 37859	 predicted : 37830.36
actual : 14995	 predicted : 14995.0
actual : 27950	 predicted : 27950.0
actual : 34990	 predicted : 34990.0
actual : 24977	 predicted : 24997.36
actual : 17333	 predicted : 14750.35
actual : 2599

# Analiza

# Selekcija (vuk)

# Modeli mašinsko učenja

## Resampling (sara)

## F-regression (sara)

## Decision Tree (mixi)

## Random forest (mixi)

## Neuronske mreže (vuk)

# Zaključak (sara)

# Literatura (sara)

 ## https://chatgpt.com/

In [ ]:
pip freeze > requirements.txt